In [1]:
import pandas as pd
import datetime as dt
import pymongo

In [2]:
file1=pd.read_csv('Business_Licenses.csv')
# file2=pd.read_csv('Sidewalk_Cafe_Permits.csv')

df1=pd.DataFrame(file1)
# df2=pd.DataFrame(file2)
df1.head()

FileNotFoundError: [Errno 2] File b'Business_Licenses.csv' does not exist: b'Business_Licenses.csv'

In [ ]:
df=df1[['ACCOUNT NUMBER','DOING BUSINESS AS NAME','ADDRESS','CITY','STATE','ZIP CODE','LICENSE DESCRIPTION',
       'BUSINESS ACTIVITY ID','BUSINESS ACTIVITY','APPLICATION TYPE','LICENSE TERM START DATE',
       'LICENSE TERM EXPIRATION DATE','DATE ISSUED', 'LICENSE STATUS','LATITUDE', 'LONGITUDE', 'LOCATION']]

df=df.loc[(df['BUSINESS ACTIVITY ID']=='775')|
          (df['BUSINESS ACTIVITY ID']=='782') |
         (df['BUSINESS ACTIVITY ID']=='735')]

df['license_id']=df.reset_index().index

In [ ]:
df['Expiration']=pd.to_datetime(df['LICENSE TERM EXPIRATION DATE']).dt.year

In [ ]:
df['is_active']=[True if value>2018 else False for value in df['Expiration']]


In [ ]:
active_df=df[df['is_active']==True].sort_values('Expiration',
                                                ascending=False).drop_duplicates(subset=['ACCOUNT NUMBER'])
active_df['active_id']=active_df.reset_index().index

In [ ]:
gb=df[df['is_active']==True].groupby('ACCOUNT NUMBER')

df3=gb.count().reset_index()

In [ ]:

active_licenses_table=active_df[['active_id','license_id','LICENSE DESCRIPTION','BUSINESS ACTIVITY ID',
                                 'BUSINESS ACTIVITY','LICENSE TERM START DATE','LICENSE TERM EXPIRATION DATE']]

licenses_table=df[['license_id','LICENSE DESCRIPTION','BUSINESS ACTIVITY ID','BUSINESS ACTIVITY','APPLICATION TYPE',
                    'DATE ISSUED']]

active_locations_table=active_df[['active_id','DOING BUSINESS AS NAME','LATITUDE','LONGITUDE','ADDRESS','CITY','STATE',
                          'ZIP CODE']]

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.restaurants_db



In [ ]:
collection = db.active_licenses
data=active_licenses_table.to_dict(orient='records')
collection.insert_many(data)

In [ ]:
collection=db.licenses
data=licenses_table.to_dict(orient='records')
collection.insert_many(data)

In [ ]:
collection=db.active_locations
data=active_locations_table.to_dict(orient='records')
collection.insert_many(data)

In [ ]:
results = db.active_licenses.find()
for result in results:
    print(result)